In [1]:
import numpy as np 
import pandas as pd 
import json
import re

from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
dataset = load_dataset("512duncanl/wh40k_novels")

Found cached dataset json (C:/Users/90530/.cache/huggingface/datasets/512duncanl___json/512duncanl--wh40k_novels-3308fde660ed265e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
df = pd.DataFrame(dataset['train'])
df.tail()

,text
4629,nce all directions were lost.\n\nThe years beg...
4630,tell what its final fate will be. From the so...
4631,me. I know it.'\n\n'You would do that?' Abadd...
4632,em the new cards he gathered from the burned c...
4633,inforce subframe structure at nine-six and nin...


In [4]:
def preprocess_data(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z0-9]', ' ', sentence)
    sentence = re.sub(' +', ' ', sentence)
    return sentence

cleaned_sentences = list(map(preprocess_data, list(df['text'].values)))
cleaned_sentences = cleaned_sentences
cleaned_sentences = ' '.join(cleaned_sentences)[:100000]

In [5]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts([cleaned_sentences])

tokenized_data = tokenizer.texts_to_sequences([cleaned_sentences])[0]

In [43]:
num_words = len(tokenizer.word_index) + 1
sentence_length = 5

input_data = []
output_data = []

for i in range(0, len(tokenized_data)-sentence_length):
    input_data.append(tokenized_data[i:i+sentence_length])
    output_data.append(tokenized_data[i+sentence_length])

input_data = np.array(input_data)
output_data = np.array(output_data)

output_data = to_categorical(output_data, num_classes=num_words)

In [44]:
print("input_data shape {}".format(input_data.shape))
print("output_data shape {}".format(output_data.shape))

input_data shape (18264, 5)
output_data shape (18264, 3779)


In [45]:
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=300, input_length=sentence_length))

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))

model.add(Dense(num_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
checkpoint = ModelCheckpoint('nex_word_prediction.h5', monitor='loss', save_best_only=True, verbose=1)

model.fit(input_data, output_data, epochs=10)

Epoch 1/10
571/571 [==============================] - 62s 93ms/step - loss: 6.9381 - accuracy: 0.0683
Epoch 2/10
571/571 [==============================] - 51s 90ms/step - loss: 6.4777 - accuracy: 0.0712
Epoch 3/10
 35/571 [>.............................] - ETA: 43s - loss: 6.3634 - accuracy: 0.0714

KeyboardInterrupt: 

In [ ]:
model.evaluate(input_data, output_data)